In [ ]:
!pip install tqdm --quiet

In [13]:
import requests
from tqdm.notebook import tqdm
import os.path
from calendar import monthrange
import time

API_KEY = "FpIG19BfekWUFWGPhb5Nl_zIMPA7Z7gI7ZQ1DESEGkk"
# API_KEY = "zw_mUBWe3F72OcpgY1j2FjnVQMULgW52uOJzvvdxT6A"

# Note the months to fetch and generate dates
months = [10, 11, 12]
dates = [f"""2021-{month:02}-{(day+1):02}""" for month in months for day in range(monthrange(2022, month)[1])]

# Show last 10 days to fetch
print(len(dates))
dates[-10:]

61


['2021-12-22',
 '2021-12-23',
 '2021-12-24',
 '2021-12-25',
 '2021-12-26',
 '2021-12-27',
 '2021-12-28',
 '2021-12-29',
 '2021-12-30',
 '2021-12-31']

In [ ]:
# Downloading real-time historical data.
!mkdir -p ./data/0-koda-gtfs-rt/vp/
!mkdir -p ./data/0-koda-gtfs-rt/tu/
# !open ./data/0-koda-gtfs-rt/tu/

headers = {'accept': 'application/octet-stream'}
for date in tqdm(dates):

    # Create parameter dictionary including key for this iteration
    params = {'date': date, 'key': API_KEY}

    # Construct filename and output path.
    # If files are already downloaded, we skip this iteration.
    file_path_vp = f"""./data/0-koda-gtfs-rt/vp/{params['date']}.7z"""
    file_path_tu = f"""./data/0-koda-gtfs-rt/tu/{params['date']}.7z"""
    if os.path.exists(file_path_vp) and os.path.exists(file_path_tu):
        print(date + " already fetched.")
        continue;

    # Trip Updates
    received = False
    while received == False:
        
        response_tu = requests.get('https://api.koda.trafiklab.se/KoDa/api/v2/gtfs-rt/sl/TripUpdates', 
                        params=params, headers=headers)
        print(type(response_tu.status_code), response_tu.status_code)
        
        if response_tu.status_code == 200: 
            received = True
            print("TU: Fetching")
            with open(file_path_tu, 'wb') as file:
                file.write(response_tu.content)
                
        if response_tu.status_code != 200:
            print("TU: Sleeping and trying again")
            time.sleep(10)

    # Vehicle Positions
    received = False
    while received == False:
        
        response_vp = requests.get('https://api.koda.trafiklab.se/KoDa/api/v2/gtfs-rt/sl/VehiclePositions', 
                                params=params, headers=headers)
        print(type(response_vp.status_code), response_vp.status_code)
        
        if response_tu.status_code != 202:
            print("VP: Sleeping and trying again")
            time.sleep(10)

        if str(response_vp.status_code) == "200":
            received = True
            print("VP: Fetching")
            with open(file_path_vp, 'wb') as file:
                file.write(response_vp.content)

  0%|          | 0/61 [00:00<?, ?it/s]

2021-11-01 already fetched.
2021-11-02 already fetched.
2021-11-03 already fetched.
<class 'int'> 200
TU: Fetching
<class 'int'> 200
VP: Sleeping and trying again
VP: Fetching
<class 'int'> 200
TU: Fetching
<class 'int'> 200
VP: Sleeping and trying again
VP: Fetching
<class 'int'> 200
TU: Fetching
<class 'int'> 200
VP: Sleeping and trying again
VP: Fetching
<class 'int'> 200
TU: Fetching
<class 'int'> 200
VP: Sleeping and trying again
VP: Fetching
<class 'int'> 200
TU: Fetching
<class 'int'> 200
VP: Sleeping and trying again
VP: Fetching
<class 'int'> 200
TU: Fetching
<class 'int'> 200
VP: Sleeping and trying again
VP: Fetching
<class 'int'> 200
TU: Fetching


In [12]:
response_tu.json()

{'error': 400,
 'message': '400 Bad Request: The date should be in the past. Data for today can only be downloaded starting tomorrow.'}

In [ ]:
# Downloading static daily historical data.
headers = {'accept': 'application/octet-stream'}

# Loop over all days from 1 to 26 (last one is not included)
for date in tqdm(dates):

    # Create parameter dictionary including key for this iteration
    params = {
        'date': date,
        'key': API_KEY,
    }

    # Construct filename and output path.
    # If file is already downloaded, we skip this iteration.
    file_path = f"""./data/0-koda-gtfs-static/{params['date']}.zip"""
    if os.path.exists(file_path):
        continue;

    # Fetch actual response
    response = requests.get('https://api.koda.trafiklab.se/KoDa/api/v2/gtfs-static/sl', 
                            params=params, headers=headers)

    # If status code is 200 (HTTP OK), save file. 
    if str(response.status_code) == "200":
        with open(file_path, 'wb') as file:
            file.write(response.content)